# DotA2

*Defense of the Ancients 2* is a MOBA developed by Valve, the sequel to the custom user map in Blizzard's *Warcraft III: The Frozen Throne*. It is one of the most popular games in all of eSports, and by far has the highest pay out pool of any game in history. *The International* (TI)is the premiere world championship Dota2 tournament, held annually. While the prize-pool changes year to year, 1st place takes home millions of dollars (8.5m in 2022, $18m in 2021, $15.6m in 2019, etc.). Leading up to TI are a series of leagues and qualifiers to determine participation.

Dota2 is known for its extreme complexity and steep learning curve. There are 124 heroes (each with usually at least 4 abilities), 200+ items, and teams consisting of 5 heroes each. Parametrically this makes for an absurd number of combinations just in player selections - the complexity skyrockets even more when you consider lane pairings, item builds, and more. The amount of information one needs to know to competetively play DotA2 is astronomical (not to mention the general skill needed to control a character, coordinate with a team, and react to your opponents!). Trying to quantify the complexity and meta within the game is a near impossible task, but we can sift through large amounts of match data in order to understand trends in how the game is played today.

In this notebook we are going to leverage match data from various professional leagues in 2023 leading up to TI2023 (Starting October 12,2023!). The website [OpenDota](https://docs.opendota.com/) has a complete API for requesting data. While it is free they do implement rate limiting (at which point they require paypent while limiting is in effect). So long as we keep our requsts below 60 calls per minute no rate limiting will be imposed (however... the data we need is going to be provided, so that we do not need to request data!).

We are going to work with match data from the following majors/professional brackets:

* [DreamLeague Season 19](https://liquipedia.net/dota2/DreamLeague/Season_19) (2023-04-09 - 2023-04-23)
* [DreamLeague Season 20](https://liquipedia.net/dota2/DreamLeague/Season_20) (2023-06-11 - 2023-06-25)
* [DreamLeague Season 21](https://liquipedia.net/dota2/DreamLeague/Season_21) (2023-09-18 - 2023-09-24)
* [Lima Major 2023](https://liquipedia.net/dota2/Lima_Major/2023) (2023-02-22 - 2023-03-05)
* [ESL One Berlin Major 2023](https://liquipedia.net/dota2/ESL_One/Berlin_Major/2023) (2023-04-26 - 2023-05-07)
* [Bali Major 2023](https://liquipedia.net/dota2/Bali_Major/2023) (2023-06-29 - 2023-07-09)


## Goals

1. Determine the most popular heroes played
2. Determine the most successful heroes played
3. Determine how successful the most popular heroes are
4. Determine the most *bannned* heroes; i.e. the heroes that teams do not wish to play against
5. Determine the success of the most banned heroes *in games where they are not banned*

### Secondary Goals

1. Learn to manipulate dataframes and explore data in tandem
2. Learn how to focus our view of the data by looking at higher-impact data and deriving new data
3. Learn how to merge multiple datasets together

## The Analysis

We need to load our data first - this code is given to us!

In [1]:
import pandas as pd

# list of files containing match data
match_list = [
    'matches_dreamleague_19.json',
    'matches_dreamleague_20.json',
    'matches_dreamleague_21.json',
    'matches_major_lima.json',
    'matches_major_berlin.json',
    'matches_major_bali.json',
]

# create a list of dataframes
match_data = [pd.read_json(m) for m in match_list]

# concatenate all of the dataframes into a single, monolithic dataframe
matches = pd.concat(
    match_data,
    ignore_index=True
)

matches.dropna(subset=['picks_bans'],inplace=True)
matches

,match_id,match_seq_num,radiant_win,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,cluster,...,radiant_gold_adv,radiant_xp_adv,teamfights,version,cosmetics,radiant_score,dire_score,draft_timings,series_id,series_type
0,7115496899,5977534560,True,2023-04-20 12:25:02,2663,1542,260,15,51,271,...,"[-312, -254, -181, 4, 832, 29, -12, -1259, -13...","[-100, -96, 229, 223, 685, -87, 229, -1240, -7...","[{'start': 571, 'end': 632, 'last_death': 617,...",21.0,"{'647': 132, '4212': 0, '4214': 0, '4215': 0, ...",48,40,"[{'order': 1, 'pick': False, 'active_team': 3,...",771259,1
1,7101545930,5965062438,True,2023-04-10 18:19:56,1793,1982,1956,63,63,273,...,"[0, -83, 72, -4, -281, -770, -418, -240, -1671...","[0, -86, 312, 179, 293, -181, 171, 837, -274, ...","[{'start': 359, 'end': 394, 'last_death': 379,...",21.0,"{'647': 131, '6646': 131, '7083': 1, '7427': 1...",37,22,"[{'order': 1, 'pick': False, 'active_team': 3,...",768596,1
2,7099579762,5963295010,True,2023-04-09 12:28:11,1947,1844,0,63,0,274,...,"[0, 629, 1064, 871, 1662, 2189, 2464, 3170, 30...","[0, 231, 148, 249, 642, 925, 1486, 2323, 2081,...","[{'start': 787, 'end': 826, 'last_death': 811,...",21.0,"{'630': 130, '647': 0, '746': 132, '4044': 132...",33,13,"[{'order': 1, 'pick': False, 'active_team': 3,...",768282,1
3,7099972924,5963673530,True,2023-04-09 16:32:01,2354,1847,0,63,0,273,...,"[-8, -399, -577, -470, -117, -85, -491, -378, ...","[-12, -41, -149, -40, 28, 290, -236, -24, -277...","[{'start': 1168, 'end': 1218, 'last_death': 12...",21.0,"{'647': 4, '4322': 128, '4476': 128, '4566': 2...",25,20,"[{'order': 1, 'pick': False, 'active_team': 3,...",768328,1
4,7100193942,5963841716,False,2023-04-09 18:56:00,1994,384,1982,51,63,273,...,"[-315, -47, -185, -225, -402, -608, -1041, -14...","[-100, -26, -204, -142, -322, -763, -1239, -14...","[{'start': 988, 'end': 1024, 'last_death': 100...",21.0,"{'647': 132, '684': 130, '5101': 131, '5644': ...",14,44,"[{'order': 1, 'pick': False, 'active_team': 3,...",768396,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,7220548968,6071023256,False,2023-06-29 08:33:53,1835,1792,1974,48,63,151,...,"[0, 347, 242, -66, 48, 71, -478, -63, 1209, 79...","[0, 370, 73, -99, -405, -246, -809, -415, 1563...","[{'start': 712, 'end': 762, 'last_death': 747,...",21.0,"{'647': 0, '7277': 0, '7580': 1, '7581': 130, ...",17,27,"[{'order': 1, 'pick': False, 'active_team': 3,...",788348,1
1058,7226053436,6075962402,False,2023-07-03 03:35:56,2395,0,2036,0,63,151,...,"[100, 18, -380, -624, -828, -1061, -1468, -152...","[70, -14, -122, -432, -256, -558, -1009, -1101...","[{'start': -13, 'end': 33, 'last_death': 18, '...",21.0,"{'647': 131, '4869': 130, '4870': 130, '4872':...",14,40,"[{'order': 1, 'pick': False, 'active_team': 3,...",789394,1
1059,7224774802,6074812509,True,2023-07-02 07:53:13,1805,1983,1572,63,15,151,...,"[-463, -113, -260, -201, -244, 164, 221, 1064,...","[-99, -162, 101, -82, 154, 96, 271, 1136, 1986...","[{'start': 1691, 'end': 1751, 'last_death': 17...",21.0,"{'647': 132, '4648': 2, '4885': 2, '5101': 0, ...",20,10,"[{'order': 1, 'pick': False, 'active_team': 3,...",789147,1
1060,7224708614,6074758569,False,2023-07-02 06:55:07,1863,1792,1974,48,63,154,...,"[0, -581, -480, -471, -438, -67, 457, 711, 685...","[0, -422, -106, 25, 51, 31, 430, -223, -166, 1...","[{'start': 1167, 'end': 1213, 'last_death': 11...",21.0,"{'647': 128, '746': 2, '5810': 130, '5919': 13...",5,31,"[{'order': 1, 'pick': False, 'active_team': 3,...",789147,1


In [2]:
matches.columns

Index(['match_id', 'match_seq_num', 'radiant_win', 'start_time', 'duration',
       'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant',
       'barracks_status_dire', 'cluster', 'first_blood_time', 'lobby_type',
       'human_players', 'leagueid', 'positive_votes', 'negative_votes',
       'game_mode', 'engine', 'picks_bans', 'radiant_team_id', 'dire_team_id',
       'radiant_team_name', 'dire_team_name', 'radiant_team_complete',
       'dire_team_complete', 'radiant_captain', 'dire_captain', 'chat',
       'objectives', 'radiant_gold_adv', 'radiant_xp_adv', 'teamfights',
       'version', 'cosmetics', 'radiant_score', 'dire_score', 'draft_timings',
       'series_id', 'series_type'],
      dtype='object')

The data is very complex and very structured. A number of columns are nested with complex structures of data. Additionally, the use of ID numbers (also known as *categoricals*) is heavy throughout. This makes the data harder to read as a human, but is much more efficient and easier to manage in code.

We want to grab the set of heroes available. Our match data uses ID numbers to represent hero selections, and so we need a way to determine which ID corresponds to which hero.

In [3]:
# read heroes.json
heroes = pd.read_json('heroes.json')
heroes

,id,name,localized_name,primary_attr,attack_type,roles,legs
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
1,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Carry]",2
2,3,npc_dota_hero_bane,Bane,all,Ranged,"[Support, Disabler, Nuker, Durable]",4
3,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Nuker, Initiator]",2
4,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker]",2
...,...,...,...,...,...,...,...
119,129,npc_dota_hero_mars,Mars,str,Melee,"[Carry, Initiator, Disabler, Durable]",2
120,135,npc_dota_hero_dawnbreaker,Dawnbreaker,str,Melee,"[Carry, Durable]",2
121,136,npc_dota_hero_marci,Marci,all,Melee,"[Support, Carry, Initiator, Disabler, Escape]",2
122,137,npc_dota_hero_primal_beast,Primal Beast,str,Melee,"[Initiator, Durable, Disabler]",2


The hero data is pretty standard - **importantly though it allows us to match ID numbers to hero names!**

Let's walk through a quick example on how we can look at the hero selections for a particular match. First we can grab the `picks_bans` data for the first match in our list. *The `picks_bans` data is yet another table!* We can take a single entry from that columne, and construct a brand new dataframe from it (as in its raw form it is still just JSON (list of dictionaries)).

In [4]:
# unpack a single record of picks_bans
# matches.picks_bans[0]
pick_bans = pd.DataFrame(matches.iloc[0].picks_bans)


This shows us whether a hero was picked or banned. We do not care much for the heros banned (for now), but we do care about which ones were picked! We also care about the team that picked them, as it will allow us to build out a dataset describing the frequency of wins for heroes throughout the professional circuit.

We can use the fact that the `is_pick` column is a *series of booleans*, which means that we can *use it as a mask!*. Let's mask our `picks_bans` from the previous cell using the `is_pick` column.

In [5]:
# mask for picks
picks = pick_bans[pick_bans.is_pick]
picks



,is_pick,hero_id,team,order
4,True,85,0,4
5,True,87,1,5
6,True,120,1,6
7,True,13,0,7
14,True,101,1,14
15,True,53,0,15
16,True,107,0,16
17,True,49,1,17
22,True,70,0,22
23,True,42,1,23


This tells us... nothing... not directly at least! This is because we do not care about hero IDs! We need to find a way to map the hero ID numbers to hero names.

We can use `pandas.merge` to merge the `picks` and `heroes` tables together. This merging process will smash our two tables together matching records in each set to one another where the hero IDs in the picks and bans table match the hero IDs from the heroes table.

In [6]:
# merge picks and heroes
named_picks = pd.merge(picks,heroes, left_on='hero_id',right_on='id')
named_picks
# named_picks.index = picks.hero_id
# named_picks.sort_values('team')

,is_pick,hero_id,team,order,id,name,localized_name,primary_attr,attack_type,roles,legs
0,True,85,0,4,85,npc_dota_hero_undying,Undying,str,Melee,"[Support, Durable, Disabler, Nuker]",2
1,True,87,1,5,87,npc_dota_hero_disruptor,Disruptor,int,Ranged,"[Support, Disabler, Nuker, Initiator]",2
2,True,120,1,6,120,npc_dota_hero_pangolier,Pangolier,all,Melee,"[Carry, Nuker, Disabler, Durable, Escape, Init...",2
3,True,13,0,7,13,npc_dota_hero_puck,Puck,int,Ranged,"[Initiator, Disabler, Escape, Nuker]",2
4,True,101,1,14,101,npc_dota_hero_skywrath_mage,Skywrath Mage,int,Ranged,"[Support, Nuker, Disabler]",2
5,True,53,0,15,53,npc_dota_hero_furion,Nature's Prophet,int,Ranged,"[Carry, Pusher, Escape, Nuker]",2
6,True,107,0,16,107,npc_dota_hero_earth_spirit,Earth Spirit,str,Melee,"[Nuker, Escape, Disabler, Initiator, Durable]",2
7,True,49,1,17,49,npc_dota_hero_dragon_knight,Dragon Knight,str,Melee,"[Carry, Pusher, Durable, Disabler, Initiator, ...",2
8,True,70,0,22,70,npc_dota_hero_ursa,Ursa,agi,Melee,"[Carry, Durable, Disabler]",2
9,True,42,1,23,42,npc_dota_hero_skeleton_king,Wraith King,str,Melee,"[Carry, Support, Durable, Disabler, Initiator]",2


In [7]:
named_picks.sort_values('hero_id')

,is_pick,hero_id,team,order,id,name,localized_name,primary_attr,attack_type,roles,legs
3,True,13,0,7,13,npc_dota_hero_puck,Puck,int,Ranged,"[Initiator, Disabler, Escape, Nuker]",2
9,True,42,1,23,42,npc_dota_hero_skeleton_king,Wraith King,str,Melee,"[Carry, Support, Durable, Disabler, Initiator]",2
7,True,49,1,17,49,npc_dota_hero_dragon_knight,Dragon Knight,str,Melee,"[Carry, Pusher, Durable, Disabler, Initiator, ...",2
5,True,53,0,15,53,npc_dota_hero_furion,Nature's Prophet,int,Ranged,"[Carry, Pusher, Escape, Nuker]",2
8,True,70,0,22,70,npc_dota_hero_ursa,Ursa,agi,Melee,"[Carry, Durable, Disabler]",2
0,True,85,0,4,85,npc_dota_hero_undying,Undying,str,Melee,"[Support, Durable, Disabler, Nuker]",2
1,True,87,1,5,87,npc_dota_hero_disruptor,Disruptor,int,Ranged,"[Support, Disabler, Nuker, Initiator]",2
4,True,101,1,14,101,npc_dota_hero_skywrath_mage,Skywrath Mage,int,Ranged,"[Support, Nuker, Disabler]",2
6,True,107,0,16,107,npc_dota_hero_earth_spirit,Earth Spirit,str,Melee,"[Nuker, Escape, Disabler, Initiator, Durable]",2
2,True,120,1,6,120,npc_dota_hero_pangolier,Pangolier,all,Melee,"[Carry, Nuker, Disabler, Durable, Escape, Init...",2


Really what we want to do here is combine the picks (and bans) data with the match results. This will let us understand how effective some heroes may be. We can see the winning team by observing the `radiant_win` column.

In [8]:
# inspect win value
matches.iloc[0].radiant_win

True

There are two teams in DotA2 - *The Radiant* and *The Dire*. There are no draws, and so the result of any match is always a Radiant win or a Dire win.

This allows the data maintainers to use a simple boolean to manage this aspect of the data - there is no `dire_win` column! What we know though is that "team 0" refers to Radiant, and "team 1" refers to Dire. All we need to do is use that boolean to assign a `win` column to the `picks_bans` table.

We can use a little trick when dealing with boolean values - we can directly compare integers and booleans *when the integers are 0 and 1*. This is very convenient for us, all we need to do is check if the `team` column in the `picks_bans` table *is not equal* to the `radiant_win` value from the match!

This gives us four combinations:

* Radiant Win (True) and hero picked by Radiant (0, or False)
* Radiant Win (True) and hero picked by Dire (1, or True)
* Radiant Loss (False) and hero picked by Radiant (0, or False)
* Radiant Loss (False) and hero picked by Dire (1, or True)

Again, note that a hero wins a match when the column indicating who won is not equal to the column indicating what team they played on.

In [9]:
# add wins column
named_picks['win'] = matches.iloc[0].radiant_win != named_picks.team
named_picks

,is_pick,hero_id,team,order,id,name,localized_name,primary_attr,attack_type,roles,legs,win
0,True,85,0,4,85,npc_dota_hero_undying,Undying,str,Melee,"[Support, Durable, Disabler, Nuker]",2,True
1,True,87,1,5,87,npc_dota_hero_disruptor,Disruptor,int,Ranged,"[Support, Disabler, Nuker, Initiator]",2,False
2,True,120,1,6,120,npc_dota_hero_pangolier,Pangolier,all,Melee,"[Carry, Nuker, Disabler, Durable, Escape, Init...",2,False
3,True,13,0,7,13,npc_dota_hero_puck,Puck,int,Ranged,"[Initiator, Disabler, Escape, Nuker]",2,True
4,True,101,1,14,101,npc_dota_hero_skywrath_mage,Skywrath Mage,int,Ranged,"[Support, Nuker, Disabler]",2,False
5,True,53,0,15,53,npc_dota_hero_furion,Nature's Prophet,int,Ranged,"[Carry, Pusher, Escape, Nuker]",2,True
6,True,107,0,16,107,npc_dota_hero_earth_spirit,Earth Spirit,str,Melee,"[Nuker, Escape, Disabler, Initiator, Durable]",2,True
7,True,49,1,17,49,npc_dota_hero_dragon_knight,Dragon Knight,str,Melee,"[Carry, Pusher, Durable, Disabler, Initiator, ...",2,False
8,True,70,0,22,70,npc_dota_hero_ursa,Ursa,agi,Melee,"[Carry, Durable, Disabler]",2,True
9,True,42,1,23,42,npc_dota_hero_skeleton_king,Wraith King,str,Melee,"[Carry, Support, Durable, Disabler, Initiator]",2,False


This is just for a single match though! What we need to do is apply this sort of processing to every match in our table. We need to create a dataframe that includes all of the hero statistics across all matches. This requires us to convert every `picks_ban` element into a  dataframe, and then concatenate all of those dataframes into a single dataframe. 

To do this, we need to define a new function 

In [10]:
# build out pick statistics
def get_hero_statistics(match_data):
    picks_bans = pd.DataFrame(match_data.picks_bans)
    picks = picks_bans[picks_bans.is_pick].drop(['is_pick'],axis=1)
    picks['match_id'] = match_data.match_id
    picks['win'] = picks.team != match_data.radiant_win
    return picks.reset_index(drop=True)

matches_heroes

Now that our hero stats table is complete, we are ready to merge it with the actual hero metadata to form a complete set of hero data.

In [11]:
# merge pick statistics with heroes

We want to look at how popular and how successful heroes are. We want to get the total number of wins, total number of picks, and average pick place (how early was a hero picked in the draft).

We can treat booleans like integers here and so simply applying a sum to the `win` column tells us how many wins that hero has. Similarly, we can just count the column to get the total number of picks. Then all we need to do is take the mean of the `order` column and we have our data!

Here we are applying the aggregation and then sorting by the number of wins (it looks a bit odd though - our column name that we are sorting by is... `('win', 'sum')`?)

In [12]:
# aggregate data

Let's fix our columns - they are not the easiest to work with and we can be a little more descriptive. We then provide the same sorting as before to view the same data:

In [13]:
# clean and sort pick data

From here we can do a lot! We can compute the win percentage of each hero:

In [14]:
# add win_pct column to pick data

Or we can sort by `avg_pick` to figure out how eagerly some heroes are selected! The lower the average pick, the earlier the hero was selected.

In [15]:
# sort pick data by the avg_pick

## Bans

Something we explicitly cut out of the above analysis is the *ban* aspect of the hero selection data. In competitive matches, not only are heroes picked, but they are also *banned*. When a hero is banned, they cannot be selected by either team. Some teams may be exceptionalyl adept at playing certain heroes and so the other team may want to remove those heroes from the game. Or perhaps a team is susceptible to a particular hero, so they ban that hero. Or perhaps a hero is known to be exceptionally powerful and difficult to play against, and so a team decides to ban that hero simply so that no one can have them.

We want to perform the above analysis, but instead of looking at picks and win percentages, we want to take a look at how often heroes are banned.

Follow along with the steps below:

Create a new data frame named `matches_banned_heroes` very similar to our `matches_heroes` data frame, except we want to only work with the heroes that have been banned! This requires us to negate `picks_bans.is_pick`. This data set also does not benefit from having a `win` column, and so we do not need to produce that for this part.

In [16]:
# get ban statistics

Next we will merge `matches_banned_heroes` with our `heroes` data frame from before just as we did when working with the previous pieces of data. Here we can just overwrite `matches_banned_heroes` with the merged data.

In [17]:
# merge with heroes

With our ban data ready we can group `matches_banned_heroes` by the `localized_name` and then apply a `count` aggregation to the `id` column. This will produce a series object with the total number of bans per hero (the heroes' localized names will appear in the index). We can save off these counts in new variable named `ban_data`.

We will also want to ensure that `ban_data` is a data frame and not a series - we can use the following to ensure it is a dataframe with the proper column names:

In [18]:
ban_data = matches_banned_heroes.groupby('localized_name').id.count().to_frame()
ban_data.columns = ['bans']
ban_data

NameError: name 'matches_banned_heroes' is not defined

Next we want to grab the 10 most banned heroes - we can do this by sorting `ban_data` descending and then taking 10 values from the `head` of the resulting data frame.

In [ ]:
# sort ban data

## Putting it all Together

Next we want to extract the performance of the 10 most banned heroes and merge that with the pick data from the first section (`pick_data`).

In [ ]:
most_banned_hero_performances = most_banned_heroes.merge(
    pick_data,
    left_index=True,
    right_index=True
)
most_banned_hero_performances

Now we can scatter this data, putting the `picks` on the x, `bans` on the y, and coloring by `win_pct` (it is also a good idea to use a different colormap (`cmap`) as the default is not the most useful).

In [ ]:
import hvplot.pandas
most_banned_hero_performances.hvplot.scatter(
    x='picks',
    y='bans',
    c='win_pct',
    hover_cols=['localized_name']
).opts(
    default_tools=['box_zoom', 'reset'],
    show_grid=True,
    width=768,
    height=512
)

This plot is fine, but it does not show us much. It would be great to look at these data points in the context of all of the other heroes in our data set. We need to merge our `pick_data` with the `ban_data` to get the complete picture. use the following code:

In [ ]:
all_picks_vs_bans = pick_data.merge(
    ban_data,
    left_index=True,
    right_index=True,
    how='left'
).fillna(0).astype({'bans': int})
all_picks_vs_bans

In the code above we are merging `pick_data` and `ban_data` by their indices. The problem here though is that there are some heroes that were never picked/banned, and so we need to account for that. After the merge is completed we end up with some values that are `NaN`, or not-a-number. This happens because there are some heroes that were picked but never banned, and so when the `bans` column is merged in, those heroes do not have ban data to incorporate. `pandas` in this case defaults them to `NaN` and converts the column to a floating point type. To get around this we need to fill the `NaN` values with 0, and then cast the `bans` column to an `int` type.

With that behind us we can now scatter plot the `picks` vs `wins`, sizing *and* coloring the points by the number of `bans`. The use of `bans` to both color and size the points gives a reinforced view of the data.

In [ ]:
# scatter all_picks_vs_bans as specified above
all_picks_vs_bans.hvplot.scatter(
    x='picks',
    y='win_pct',
    c='bans',
    s='bans',
    hover_cols=['localized_name'],
).opts(
    default_tools=['box_zoom', 'reset'],
    show_grid=True,
    width=768,
    height=512
)